In [ ]:
import threading
import datetime
import time
import numpy as np
import cv2 
import socket 
from queue import Queue, Empty
import keyboard
#Подключение собственных модулей:
from sender import Sender_SOCKET
from camera import Videocamera
from detector import detect_cloude, viewImage

In [15]:
if __name__ == '__main__':#Если файл является исполняемым
    queue_images = Queue() #Очередь для изображений с камеры
    queue_segments_of_images = Queue() #Очередь для сегментов изображений
    history_of_results_detecting = list() #История результатов обработки
    camera = Videocamera(1, 'http://192.168.0.100:8080/video', queue_images)
    sender = Sender_SOCKET()
    while True :
        if keyboard.is_pressed('q'):#Кнопка для выхода из цикла
            break
        results_detecting = list()
        thread_main = threading.Thread(target=camera.get_frame, args=[], daemon = True)
        thread_main.start() 
        try:
            image = np.array(queue_images.get())
        except:
            break
        center_x = image.shape[0]//2
        center_y = image.shape[1]//2
        #Сегментирование кадра
        queue_segments_of_images.put_nowait(image[0:center_x, 0:center_y])
        queue_segments_of_images.put_nowait(image[center_x: , center_y: ])
        queue_segments_of_images.put_nowait(image[0:center_x, center_y: ])
        queue_segments_of_images.put_nowait(image[center_x: , 0:center_y:])

        thread_workers =[
            threading.Thread(target= detect_cloude, args = [queue_segments_of_images, results_detecting], name = 'a', daemon = True), 
            threading.Thread(target= detect_cloude, args = [queue_segments_of_images, results_detecting], name = 'b', daemon = True),
            threading.Thread(target= detect_cloude, args = [queue_segments_of_images, results_detecting], name = 'c', daemon = True),
            threading.Thread(target= detect_cloude, args = [queue_segments_of_images, results_detecting], name = 'd', daemon = True)
        ]

        for thread in thread_workers:
            thread.start()
            time.sleep(1)
        thread_main.join()
        for thread in thread_workers:
            thread.join()
            print("Thread {} complete it's works".format(thread.name))
        total_area = sum(results_detecting)
        if total_area>13000:
            sender.send('{} ;Dull; {}'.format(camera.id, total_area))
        elif 2000<total_area<13000:
            sender.send('{} ;Cloudy; {}'.format(camera.id, total_area))
        else:
            sender.send('{} ;Sun; {}'.format(camera.id, total_area))
        history_of_results_detecting.extend(results_detecting)
        print('*'*30)
        time.sleep(20)
    camera.stop()
    sender.close()


Thread a complete it's works
Thread b complete it's works
Thread c complete it's works
Thread d complete it's works
******************************
Thread a complete it's works
Thread b complete it's works
Thread c complete it's works
Thread d complete it's works
******************************
Thread a complete it's works
Thread b complete it's works
Thread c complete it's works
Thread d complete it's works
******************************
Thread a complete it's works
Thread b complete it's works
Thread c complete it's works
Thread d complete it's works
******************************
Thread a complete it's works
Thread b complete it's works
Thread c complete it's works
Thread d complete it's works
******************************
Thread a complete it's works
Thread b complete it's works
Thread c complete it's works
Thread d complete it's works
******************************
Thread a complete it's works
Thread b complete it's works
Thread c complete it's works
Thread d complete it's works
**

IndexError: tuple index out of range